# Estimation problems

In this tutorial, we explain one of the greatest features of GeoStats.jl: the ability to setup geostatistical problems indenpendently of the solution strategy.

If you are an experienced user of geostatistics or if you do research in the field, you know how hard it is to compare algorithms fairly. Often a new algorithm is proposed in the literature, and yet the task of comparing it with the state of the art is quite demanding. Even when a comparison is made by the author after a great amount of effort, it is inevitably biased.

Part of this issue is attributed to the fact that a general definition of the problem is missing. What is it that we call an "estimation problem" in geostatistics? The answer to this question is given below in the form of code.

Before we proceed, please install the following packages:

In [1]:
for pkg in ["GeoStats", "Plots", "PyPlot"]
    Pkg.add(pkg)
end

# make sure this tutorial is reproducible
srand(2000);

INFO: Package GeoStats is already installed
INFO: Package Plots is already installed
INFO: Package PyPlot is already installed


## Problem definition

An estimation problem in geostatistics is a triplet:

1. Spatial data (i.e. data with coordinates)
2. Spatial domain (e.g. regular grid, unstructured grid)
3. Target variables (or variables to be estimated)

It doesn't involve variograms, training images, or any tuning parameter. These concepts belong to solvers.

Let's make it a concrete example, and create some data:

In [2]:
using GeoStats

geodata = readtable("data/precipitation.csv", coordnames=[:x,:y])

3×4 GeoDataFrame (x and y)

│ Row │ x    │ y    │ station       │ precipitation │
├─────┼──────┼──────┼───────────────┼───────────────┤
│ 1   │ 25.0 │ 25.0 │ palo alto     │ 1.0           │
│ 2   │ 50.0 │ 75.0 │ redwood city  │ 0.0           │
│ 3   │ 75.0 │ 50.0 │ mountain view │ 1.0           │

The `GeoDataFrame` type is defined in GeoStats.jl as a lightweight wrapper over Julia's DataFrame types. Besides the data, it stores the name of the columns representing spatial coordinates. In this example, we used the `readtable` utility function to read the data directly from disk.

Next, we define the domain in which the variables will be estimated. One of the many choices possible is the regular grid:

In [3]:
domain = RegularGrid{Float64}(100,100)

RegularGrid{Float64,2}
  dimensions: (100, 100)
  origin:     (0.0, 0.0)
  spacing:    (1.0, 1.0)

Notice that by default, a regular grid has zero origin and unit spacing. Alternatively, these parameters can be specified explicitly:

In [4]:
domain = RegularGrid([100,100], [0.,0.], [1.,1.])

RegularGrid{Float64,2}
  dimensions: (100, 100)
  origin:     (0.0, 0.0)
  spacing:    (1.0, 1.0)

Like many other domain types in GeoStats.jl, regular grids are lightweight objects. They do not require any memory space other than the space used to save the input parameters (i.e. dimensions, origin and spacing). Looping over a regular grid or finding the coordinates of a given location is done analytically for maximum performance.

Finally, we define the estimation problem for the precipitation variable:

In [5]:
problem = EstimationProblem(geodata, domain, :precipitation)

2D EstimationProblem
  data:      3×4 GeoDataFrame (x and y)
  domain:    100×100 RegularGrid{Float64,2}
  variables: precipitation (Float64)

## Solving the problem

Now that the problem is unambiguously defined, we can solve it with various estimation solvers. In this tutorial, we will use the polyalgorithm Kriging solver distributed with GeoStats.jl. In Kriging, each variable of the problem is (optionally) parametrized by a mean and a variogram:

In [6]:
solver = Kriging(
    :precipitation => @NT(variogram=GaussianVariogram(range=35.))
)

GeoStats.Kriging
  └─precipitation
    └─variogram = Variography.GaussianVariogram{Float64,Float64,Distances.Euclidean}
  range: Float64 35.0
  sill: Float64 1.0
  nugget: Float64 0.0
  distance: Distances.Euclidean

    └─mean = nothing
    └─degree = nothing
    └─drifts = nothing


The line above translates to "solve the precipitation variable using a Gaussian variogram". The prefix `@NT` extends for `NamedTuple` and won't be necessary in Julia v0.7 and beyond. When only the variogram is specified, Ordinary Kriging is triggered.

The user can specify the mean (e.g. `mean=.5`) for Simple Kriging, the polynomial degree (e.g. `degree=1`) for Universal Kriging, and the drift functions (e.g. `drifts=[x -> 1 + x[1], x -> 2x[2]]`) for External Drift Kriging. For more solver options, please consult the GeoStats.jl documentation.

The solution to the problem is easily obtained with:

In [7]:
solution = solve(problem, solver)

2D EstimationSolution
  domain: 100×100 RegularGrid{Float64,2}
  variables: precipitation

It is stored in an efficient format with all the necessary information to reconstruct the estimates spatially. The user can digest the solution into a Julia dictionary by calling `digest(solution)`. The resulting dictionary maps variable names (e.g. :precipitation) to their mean and variance:

In [8]:
digest(solution)

Dict{Symbol,Dict{Symbol,Array{Float64,2}}} with 1 entry:
  :precipitation => Dict(:mean=>[0.687121 0.689082 … 0.671328 0.671373; 0.68908…

However, very often we just want to visualize the results. In GeoStats.jl, solutions to estimation problems can be plotted directly in a standardized format for comparison between different solvers and parameter settings:

In [9]:
using Plots; pyplot()

plot(solution, size=(1000,500))

INFO: Recompiling stale cache file /home/juliohm/.julia/lib/v0.6/PyCall.ji for module PyCall.
INFO: Recompiling stale cache file /home/juliohm/.julia/lib/v0.6/PyPlot.ji for module PyPlot.


## Conclusions

The ability to work at the level of the problem definition is quite desirable. Users can switch between different solvers without having to learn new syntax. This approach also guarantees that the problem being solved is the same and that the comparison is fair.

Documentation on how to write estimation solvers can be found in the [GeoStatsBase.jl](https://github.com/juliohm/GeoStatsBase.jl) package. Additional helper tools for developers can be found in [GeoStatsDevTools.jl](https://github.com/juliohm/GeoStatsDevTools.jl). Please open an issue in any of these modules if you have questions about solver development.